<p>
The purpose of this article is to illuminate how Java is installed and accessed on Mac OS X 10.5 versions and later. If you execute <code>which java</code> this will output <code>/usr/bin/java</code>.
</p>

<p>
This is not where Java resides, it is a symbolic link to <code>/System/Library/Frameworks/JavaVM.framework/Versions/Current/Commands/java</code>
</p>

<p>
And confusingly enough, this is also not where Java resides. Files contained in <code>/System</code> tend to be Apple specific and not 3rd party. The second clue, as pointed out by Stack Overflow user <i>bdash</i> <a href="#ref1">[1]</a>, are files in: <code>/System/Library/Frameworks/JavaVM.framework/Versions/Current/Commands</code> are on the order of 10 bytes in size. The real Java executable is bigger than that!

<h1>Dynamic Tracing</h1>

<i>bdash</i> gives the following command to run</a>:
<code>sudo dtrace -n 'syscall::posix_spawn:entry { trace(copyinstr(arg1)); }' -c "/usr/bin/java -version"</code>

This outputs:
<p>
<code>
dtrace: description 'syscall::posix_spawn:entry' matched 1 probe
dtrace: pid 712 has exited
CPU   ID   FUNCTION:NAME
  0  629  posix_spawn:entry  /Library/Java/JavaVirtualMachines/jdk1.7.0_65.jdk/Contents/Home/bin/java
</code>
</p>
<p>
This suggests the Java program resides in
</p>
<p>
<code>/Library/Java/JavaVirtualMachines/jdk1.7.0_65.jdk/Contents/Home/bin/java</code>
</p>
<p>
and is indeed correct. Also, to confirm that I have one JDK version, I observe that my directory <code>/Library/Java/JavaVirtualMachines</code> only contains one directory named <code>jdk1.7.0_65.jdk</code> Multiple JDK versions are allowed and will reside in <code>/Library/Java/JavaVirtualMachines/</code>
</p>
<p>
The next question is, how are multiple versions managed? And the answer is <code>java_home</code>
</p>

<h1>Java Home</h1>

Since Mac OS X 10.5, Apple has provided a tool called <code>java_home</code>. Its function is to dynamically find the Java version a user specifies in Java Preferences <a href="#ref2">[2]</a>. It returns the home path of the current JDK in use. I run this program with help flag:

<code>/usr/libexec/java_home --help</code>

And discover the <code>--verbose</code> flag. It tells me the full list of JVM architectures I have on my machine. When I run this with this flag on my machine (Macbook OS X 10.10.3) I get the following output:

<p>
<code>
Matching Java Virtual Machines (1):
   1.7.0_65, x86_64: "Java SE 7" /Library/JavaVirtualMachines/jdk1.7.0_65.jdk/Contents/Home
</code>
</p>

This confirms what I observed earlier in <code>/Library/Java/JavaVirtualMachines</code>, that I only have one JDK version. Just like <code>/usr/bin/java</code>, <code>java_home</code> is a symbolic link to a file that resides in:

<code>/System/Library/Frameworks/JavaVM.framework/Versions/Current/Commands</code>


By the way, <code>/usr/libexec</code> is supposed to a folder that contains binaries meant to be run by other programs (i.e., not users) <a href="#ref3">[3]</a>.

<h1>Java Applet</h1>

There's one last Java version sitting around that should be explained as well. Consider:

<code>/Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home/bin/java</code>

It's not a symbolic link nor a binary that redirects to another file. What is it? The path makes it clear: it's used by Safari when it executes a Java applet. On my machine, it has a newer build. It's a JRE and to the best of my knowledge, it only allows one JRE version to be installed.

<h1>JAVA_HOME</h1>

<p>
Now that we better understand how Java is accessed on the Mac, we'd like to set the environmental variable <code>JAVA_HOME</code>. Modify your <code>~/.bash_profile</code> file:
</p>

<p>
<code>
JAVA_HOME=&#36;(/usr/libexec/java_home)

PATH=&#36;PATH:&#36;JAVA_HOME/bin

export JAVA_HOME
</code>
</p>

<p>
Using <code>java_home</code> instead of hard-coding a path, will return the path of the Java home directory. We add it to our path variable and finally, export the <code>JAVA_HOME</code> for use as an environmental variable
</p>

<p>
As a final note, apparently for versions earlier than 10.5, Apple recommends using the fixed path of <code>/Library/Java/Home</code> <a href="#ref2">[2]</a>.
</p>

<h1>References</h1>

<a id="ref1"></a>[1] http://stackoverflow.com/questions/15120745/need-help-understanding-oracles-java-on-mac/15133344#15133344

<a id="ref2"></a>[2] https://developer.apple.com/library/mac/qa/qa1170/_index.html

<a id="ref3"></a>[3] http://www.linuxbase.org/betaspecs/fhs/fhs/ch04s07.html